**Скачивание файлов**



In [ ]:
import pandas as pd

In [ ]:
!wget https://aiijc.com/api_v2/task/1066/53 -O dffolder.zip
!unzip /content/dffolder.zip

--2022-04-09 09:03:57--  https://aiijc.com/api_v2/task/1066/53
Resolving aiijc.com (aiijc.com)... 188.93.56.95
Connecting to aiijc.com (aiijc.com)|188.93.56.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26391772 (25M) [application/zip]
Saving to: ‘dffolder.zip’

dffolder.zip        100%[===================>]  25.17M  10.6MB/s    in 2.4s    

2022-04-09 09:04:02 (10.6 MB/s) - ‘dffolder.zip’ saved [26391772/26391772]

Archive:  /content/dffolder.zip
replace __MACOSX/._rabotaru_en? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/q3OBoq2qpSp32w -O parsed.csv #скачивание данных парсинга с headhunter, trudvsem и superjob

--2022-09-27 07:11:38--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/q3OBoq2qpSp32w
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/29d9ea034ebe24153768150374442b9ac7d870cff7d0be45f71c9306c38ccf9d/6332da6b/PKiwL9hDtrnduXgl_ZIG1C8ssSpdNbyM71kFjUgmFsBwFAJWpDrSGKECioJ5dBPZcOgKlfrIJiz3ZHsH2DTCWQ%3D%3D?uid=0&filename=train_.csv&disposition=attachment&hash=ca713sco/%2BRu90Ew1GlE26aHO1MTAVMtxKwI6IcbnvGNIPNk8GYb50iv4kxtmTpxq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Foctet-stream&owner_uid=1442255699&fsize=827939256&hid=f81d4237f1d0ba3d35e968b2a4107229&media_type=spreadsheet&tknv=v2 [following]
--2022-09-27 07:11:39--  https://downloader.disk.yandex.ru/disk/29d9ea034ebe24153768150374442b9ac7d870cff7d0be45f71c9306c38ccf9d/6332da6b/PKiwL9hDtrnduXgl_Z

In [ ]:
parsed = pd.read_csv('/content/parsed.csv').sample(frac=1)[70000:100000] #загрузка данных парсинга
sample = pd.read_csv('/content/rabotaru_en/data/sample.csv')
test = pd.read_csv('/content/rabotaru_en/data/test.csv')
train = pd.read_csv('/content/rabotaru_en/data/train_public.csv')

**Импортирование нужных библиотек**

In [ ]:
!pip install pymorphy2
!pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 8.2 MB 52.9 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a1b1d1166087dc485c10acad0a375f000919dc60f9d0cb043146ca6c9ebcc397
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [ ]:
import pymorphy2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
 
nltk.download('stopwords')
 
import os
import re
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from nltk import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
a = train[:5]
a

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof
0,268842,Подсобный рабочий,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,34000,0,NaN,NaN,True,True,1.0,41000.0,True,[],{1783},1234787.0,70,рабочий
1,19090,Разнорабочий,7,<br />\nТребования: <br>На овощную базу в Подм...,66000,0,NaN,NaN,False,False,NaN,NaN,NaN,[],{3},NaN,1,разнорабочий
2,315832,Повар холодного цеха,1,<br />\nТребования: <br>Приготовление салатов....,40000,0,NaN,NaN,False,False,NaN,40000.0,NaN,[],{3},NaN,1,повар
3,365230,Разнорабочий,7,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,35000,0,NaN,NaN,False,True,-100.0,40000.0,NaN,[],{5807},2784163.0,70,разнорабочий
4,154105,Региональный торговый представитель по Смоленс...,1,<strong>Обязанности:</strong><br /> &nbsp;<br ...,30000,4,NaN,NaN,False,True,2.0,70000.0,NaN,[],{362},1703984.0,258,представитель


In [ ]:
a.to_csv('kapusta.csv')

**Обработка данных**

In [ ]:
stopwords = stopwords.words('russian') #загрузка стоп-слов русского языка

In [ ]:
stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def lemmize(text): #функция лемматизации текста
  words = text.split()
  words_splitted = []
  for i in words:
    words_splitted.extend(i.split('-'))
  res = list()
  for word in words_splitted:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
  return ' '.join(res).strip()

In [ ]:
train['custom_position'] = train['custom_position'].str.lower().apply(lemmize) #леммитизация текста в custom_position
test['custom_position'] = test['custom_position'].str.lower().apply(lemmize)

In [ ]:
parsed['name'] = parsed['name'].str.lower().apply(lemmize)

In [ ]:
def remove_stopwords(text): #функция удаления стоп-слов
  words = text.split()
  new_words = []
  for i in words:
    if i not in stopwords:
      new_words.append(i)

  return ' '.join(new_words)

In [ ]:
train['custom_position'] = train['custom_position'].str.lower().apply(remove_stopwords) #удаление стоп-слов
test['custom_position'] = test['custom_position'].str.lower().apply(remove_stopwords)
parsed['name'] = parsed['name'].str.lower().apply(remove_stopwords)

In [ ]:
train.head()

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof
0,268842,подсобный рабочий,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,34000,0,NaN,NaN,True,True,1.0,41000.0,True,[],{1783},1234787.0,70,рабочий
1,19090,разнорабочий,7,<br />\nТребования: <br>На овощную базу в Подм...,66000,0,NaN,NaN,False,False,NaN,NaN,NaN,[],{3},NaN,1,разнорабочий
2,315832,повар холодное цех,1,<br />\nТребования: <br>Приготовление салатов....,40000,0,NaN,NaN,False,False,NaN,40000.0,NaN,[],{3},NaN,1,повар
3,365230,разнорабочий,7,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,35000,0,NaN,NaN,False,True,-100.0,40000.0,NaN,[],{5807},2784163.0,70,разнорабочий
4,154105,региональный торговый представитель смоленск о...,1,<strong>Обязанности:</strong><br /> &nbsp;<br ...,30000,4,NaN,NaN,False,True,2.0,70000.0,NaN,[],{362},1703984.0,258,представитель


In [ ]:
test.head()

,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,is_male,driving_license,region_id,company_id,city_id
0,286961,водитель курьер доставка еда,3,<strong>Приглашаем на работу ВОДИТЕЛЕЙ-КУРЬЕРО...,0,18.0,50.0,False,True,-100.0,NaN,['B'],{209},1172596.0,57
1,423834,упаковщик печение (вахта),7,"Прямой работодатель, с сезонным расширением пр...",0,NaN,NaN,False,False,NaN,NaN,[],{3},1879927.0,1
2,336464,менеджер туризм,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,4,NaN,NaN,False,False,1.0,NaN,[],{1278},2724192.0,57
3,287634,повар сушистый,1,Компания PIZZASUSHIWOK приглашает на работу по...,0,18.0,50.0,False,True,-100.0,NaN,[],{3},1172596.0,1
4,174883,уборщик/уборщица азс 145,3,<br /> <br />Требуется уборщик/ца на АЗС 145.<...,0,NaN,NaN,False,True,-1.0,NaN,[],{1278},1952643.0,1


In [ ]:
parsed.head()

,Unnamed: 0,description,name,label
288796,232499,<p><strong><em>Мы занимаемся разработкой и раз...,руководитель партнёрский направление сра,руководитель
155071,98774,<strong>Обязанности:</strong> <ul> <li> <p>Реш...,сотрудник выездной технический поддержка / каб...,сотрудник
129276,72979,<p><strong>Обязанности:</strong></p> <ul> <li>...,швея (трикотаж),швея
87475,31178,<p><strong><em>Группа компаний &quot;Русский Х...,автослесарь (слесарить ремонтник),слесарь
238728,182431,<strong>Обязанности:</strong> <ul> <li>Своевре...,логист,логист


**Предсказание специальностей**

In [ ]:
#X, y = list(pd.concat([train['custom_position'], parsed['name']])), list(pd.concat([train['target_prof'], parsed['label']]))
X, y = train['custom_position'], train['target_prof']
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)

In [ ]:
y_train

10962            повар
5359            курьер
51313           мастер
41153         менеджер
8748         инспектор
             ...      
42036     посудомойщик
39489           курьер
8697          оператор
8405     администратор
41616      дезинфектор
Name: target_prof, Length: 45037, dtype: object

In [ ]:
vectorizer = TfidfVectorizer().fit(X_train)

In [ ]:
import pickle
with open('vectorizer.pickle', 'wb') as f:
  pickle.dump(vectorizer, f)



In [ ]:
X_tfidf_train = np.array(vectorizer.transform(X_train).todense(), dtype=np.float16)
X_tfidf_val = np.array(vectorizer.transform(X_val).todense(), dtype=np.float16)

In [ ]:
def evaluate_model(model, X, y, print_report=False): #функция проверки качества модели
  y_pred = model.predict(X)
  f1 = f1_score(y, y_pred, average='macro')
  print(f'Macro F1 score: {f1}')
  if print_report:
    print(f'Report for model: {str(model)}')
    print(classification_report(y, y_pred, zero_division=0))
  return f1

In [ ]:
modelq = RandomForestClassifier(n_estimators = 5, n_jobs = -1).fit(X_tfidf_train, y_train) #обучение модели
evaluate_model(modelq, X_tfidf_train, y_train)
print('Validation')
evaluate_model(modelq, X_tfidf_val, y_val)

Macro F1 score: 0.9449210181981473
Validation
Macro F1 score: 0.8102386061308381


0.8102386061308381

In [ ]:
with open('/content/drive/MyDrive/npk/model5.pickle', 'wb') as f:
  pickle.dump(modelq, f)

In [ ]:
test_tfidf = np.array(vectorizer.transform(test['custom_position']).todense(), dtype=np.float16)

In [ ]:
test['target_prof'] = modelq.predict(test_tfidf)

In [ ]:
test

,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,is_male,driving_license,region_id,company_id,city_id,target_prof
0,286961,водитель курьер доставка еда,3,<strong>Приглашаем на работу ВОДИТЕЛЕЙ-КУРЬЕРО...,0,18.0,50.0,False,True,-100.0,NaN,['B'],{209},1172596.0,57,водитель
1,423834,упаковщик печение (вахта),7,"Прямой работодатель, с сезонным расширением пр...",0,NaN,NaN,False,False,NaN,NaN,[],{3},1879927.0,1,упаковщик
2,336464,менеджер туризм,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,4,NaN,NaN,False,False,1.0,NaN,[],{1278},2724192.0,57,менеджер
3,287634,повар сушистый,1,Компания PIZZASUSHIWOK приглашает на работу по...,0,18.0,50.0,False,True,-100.0,NaN,[],{3},1172596.0,1,повар
4,174883,уборщик/уборщица азс 145,3,<br /> <br />Требуется уборщик/ца на АЗС 145.<...,0,NaN,NaN,False,True,-1.0,NaN,[],{1278},1952643.0,1,уборщик
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14069,290923,маркировщик/ца детский товар,7,<strong>Работа вахтой на производственно-склад...,0,NaN,NaN,False,False,NaN,NaN,[],{3},1709808.0,1,маркировщик
14070,175080,упаковщик чайный фабрика (вахта),7,На чайную фабрику в Москве требуются сотрудник...,0,NaN,NaN,False,False,NaN,NaN,[],{3},1724853.0,1,упаковщик
14071,451820,главный бухгалтер тэо,6,Стабильно работающая более 10 лет компания ище...,4,NaN,NaN,False,True,3.0,NaN,[],{4},2961361.0,2,бухгалтер
14072,257668,помощник руководитель,1,<br />\nТребования: <br>В связи с формирование...,0,NaN,NaN,False,False,NaN,NaN,[],{3},NaN,1,ассистент


**Предсказание зарплат**

In [ ]:
test['salary_from'] = 0
test['salary_to'] = 0

In [ ]:
for i in range(len(train)): #обработка регионов
  r = str(train['region_id'].iloc[i])
  if r[0] == '{':
    r = r[1:]
  if r[-1] == '}':
    r = r[:-1]
  r = int(r.split(', ')[0])
  train.loc[i, 'region_id'] = r

for i in range(len(test)):
  r = str(test['region_id'].iloc[i])
  if r[0] == '{':
    r = r[1:]
  if r[-1] == '}':
    r = r[:-1]
  r = int(r.split(', ')[0])
  test.loc[i, 'region_id'] = r

In [ ]:
train

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof
0,268842,подсобный рабочий,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,34000,0,NaN,NaN,True,True,1.0,41000.0,True,[],1783,1234787.0,70,рабочий
1,19090,разнорабочий,7,<br />\nТребования: <br>На овощную базу в Подм...,66000,0,NaN,NaN,False,False,NaN,NaN,NaN,[],3,NaN,1,разнорабочий
2,315832,повар холодное цех,1,<br />\nТребования: <br>Приготовление салатов....,40000,0,NaN,NaN,False,False,NaN,40000.0,NaN,[],3,NaN,1,повар
3,365230,разнорабочий,7,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,35000,0,NaN,NaN,False,True,-100.0,40000.0,NaN,[],5807,2784163.0,70,разнорабочий
4,154105,региональный торговый представитель смоленск о...,1,<strong>Обязанности:</strong><br /> &nbsp;<br ...,30000,4,NaN,NaN,False,True,2.0,70000.0,NaN,[],362,1703984.0,258,представитель
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56292,441432,логист,1,<strong>Обязанности:</strong>&nbsp;<br />\r\n<...,80000,0,NaN,NaN,False,False,2.0,NaN,NaN,[],4,2915102.0,2,логист
56293,20526,кондитер,7,В Дивизионе комплексного обслуживания удаленны...,0,2,NaN,NaN,False,True,3.0,73000.0,NaN,[],526,1225566.0,18,повар
56294,280515,сборщик кожгалантерейный изделие,1,<br />\nТребования: <br>с опытом,-1,0,NaN,NaN,False,False,NaN,-1.0,NaN,[],4,NaN,2,сборщик
56295,237213,инженер связь,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,-1,4,NaN,NaN,False,True,2.0,-1.0,NaN,[],1626,1879891.0,29,инженер


In [ ]:
regs = []
for i in train['region_id']:
  if i not in regs:
    regs.append(i)
reg_median_salary = []
for i in regs:
  a = train[train['region_id'] == i][train['salary_from'] > 0]['salary_from']
  if len(a) == 0:
    reg_median_salary.append(train[train['salary_from'] > 0]['salary_from'].median())
  else:
    reg_median_salary.append(a.median())
reg_median_salary
max_i = max(reg_median_salary) #разделяем на 40 зарплатных классов регионов
for i in range(len(regs)):
  reg_median_salary[i] = round((reg_median_salary[i]/max_i)*40)
for i in range(len(train)): #присваиваем каждой строке один из зарплатных классов
  train.loc[i, 'region_id_i'] = reg_median_salary[regs.index(train['region_id'].iloc[i])]
train
for i in range(len(test)): #присваиваем каждой строке один из зарплатных классов
  try:
    test.loc[i, 'region_id_i'] = reg_median_salary[regs.index(test['region_id'].iloc[i])]
  except:
    test.loc[i, 'region_id_i'] = 18
train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof,region_id_i
0,268842,подсобный рабочий,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,34000,0,NaN,NaN,True,True,1.0,41000.0,True,[],1783,1234787.0,70,рабочий,5.0
1,19090,разнорабочий,7,<br />\nТребования: <br>На овощную базу в Подм...,66000,0,NaN,NaN,False,False,NaN,NaN,NaN,[],3,NaN,1,разнорабочий,6.0
2,315832,повар холодное цех,1,<br />\nТребования: <br>Приготовление салатов....,40000,0,NaN,NaN,False,False,NaN,40000.0,NaN,[],3,NaN,1,повар,6.0
3,365230,разнорабочий,7,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,35000,0,NaN,NaN,False,True,-100.0,40000.0,NaN,[],5807,2784163.0,70,разнорабочий,5.0
4,154105,региональный торговый представитель смоленск о...,1,<strong>Обязанности:</strong><br /> &nbsp;<br ...,30000,4,NaN,NaN,False,True,2.0,70000.0,NaN,[],362,1703984.0,258,представитель,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56292,441432,логист,1,<strong>Обязанности:</strong>&nbsp;<br />\r\n<...,80000,0,NaN,NaN,False,False,2.0,NaN,NaN,[],4,2915102.0,2,логист,5.0
56293,20526,кондитер,7,В Дивизионе комплексного обслуживания удаленны...,0,2,NaN,NaN,False,True,3.0,73000.0,NaN,[],526,1225566.0,18,повар,10.0
56294,280515,сборщик кожгалантерейный изделие,1,<br />\nТребования: <br>с опытом,-1,0,NaN,NaN,False,False,NaN,-1.0,NaN,[],4,NaN,2,сборщик,5.0
56295,237213,инженер связь,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,-1,4,NaN,NaN,False,True,2.0,-1.0,NaN,[],1626,1879891.0,29,инженер,3.0


In [ ]:
with open('regs.pickle', 'wb') as f:
  pickle.dump(regs, f)
with open('med_s_reg.pickle', 'wb') as f:
  pickle.dump(reg_median_salary, f)

In [ ]:
profs = []
for i in train['target_prof']:
  if i not in profs:
    profs.append(i)
profs_median_salary = []
for i in profs:
  a = train[train['target_prof'] == i][train['salary_from'] > 0]['salary_from']
  if len(a) == 0:
    profs_median_salary.append(train[train['salary_from'] > 0]['salary_from'].median())
  else:
    profs_median_salary.append(a.median())
max_i = max(profs_median_salary) #разделяем на 15 зарплатных классов специализаций
for i in range(len(profs)):
  profs_median_salary[i] = round((profs_median_salary[i]/max_i)*15)
for i in range(len(train)): #присваиваем каждой строке один из зарплатных классов
  train.loc[i, 'sal_class_tp'] = profs_median_salary[profs.index(train['target_prof'].iloc[i])]
train
for i in range(len(test)): #присваиваем каждой строке один из зарплатных классов
  test.loc[i, 'sal_class_tp'] = profs_median_salary[profs.index(test['target_prof'].iloc[i])]
test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,is_male,driving_license,region_id,company_id,city_id,target_prof,salary_from,salary_to,region_id_i,sal_class_tp
0,286961,водитель курьер доставка еда,3,<strong>Приглашаем на работу ВОДИТЕЛЕЙ-КУРЬЕРО...,0,18.0,50.0,False,True,-100.0,NaN,['B'],209,1172596.0,57,водитель,0,0,5.0,5.0
1,423834,упаковщик печение (вахта),7,"Прямой работодатель, с сезонным расширением пр...",0,NaN,NaN,False,False,NaN,NaN,[],3,1879927.0,1,упаковщик,0,0,6.0,4.0
2,336464,менеджер туризм,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,4,NaN,NaN,False,False,1.0,NaN,[],1278,2724192.0,57,менеджер,0,0,5.0,3.0
3,287634,повар сушистый,1,Компания PIZZASUSHIWOK приглашает на работу по...,0,18.0,50.0,False,True,-100.0,NaN,[],3,1172596.0,1,повар,0,0,6.0,3.0
4,174883,уборщик/уборщица азс 145,3,<br /> <br />Требуется уборщик/ца на АЗС 145.<...,0,NaN,NaN,False,True,-1.0,NaN,[],1278,1952643.0,1,уборщик,0,0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14069,290923,маркировщик/ца детский товар,7,<strong>Работа вахтой на производственно-склад...,0,NaN,NaN,False,False,NaN,NaN,[],3,1709808.0,1,маркировщик,0,0,6.0,3.0
14070,175080,упаковщик чайный фабрика (вахта),7,На чайную фабрику в Москве требуются сотрудник...,0,NaN,NaN,False,False,NaN,NaN,[],3,1724853.0,1,упаковщик,0,0,6.0,4.0
14071,451820,главный бухгалтер тэо,6,Стабильно работающая более 10 лет компания ище...,4,NaN,NaN,False,True,3.0,NaN,[],4,2961361.0,2,бухгалтер,0,0,5.0,3.0
14072,257668,помощник руководитель,1,<br />\nТребования: <br>В связи с формирование...,0,NaN,NaN,False,False,NaN,NaN,[],3,NaN,1,ассистент,0,0,6.0,3.0


In [ ]:
X, y = train['custom_position'], train['sal_class_tp']
vectorizer = TfidfVectorizer().fit(X_train)
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)
X_tfidf_train = np.array(vectorizer.transform(X_train).todense(), dtype=np.float16)
X_tfidf_val = np.array(vectorizer.transform(X_val).todense(), dtype=np.float16)
model = RandomForestClassifier(n_estimators = 35, n_jobs = -1, max_depth = 250).fit(X_tfidf_train, y_train) #обучение модели
evaluate_model(model, X_tfidf_train, y_train)
print('Validation')
evaluate_model(model, X_tfidf_val, y_val)

Macro F1 score: 0.7918178348418804
Validation
Macro F1 score: 0.8726578275271879


0.8726578275271879

In [ ]:
import pickle
with open('model2.pickle', 'wb') as f:
  pickle.dump(model, f)

In [ ]:
test_tfidf = np.array(vectorizer.transform(test['custom_position']).todense(), dtype=np.float16)
test['sal_class_tp'] = model.predict(test_tfidf)

In [ ]:
import re #функции, которые ищут в описании зарплату
def clear(s):
  d = list(map(str, range(10)))
  m1 = []
  for i in s:
    if i != ' ':
      m1.append(i)
  s = ''.join(m1)
  m = []
  for i in s:
    if i in d:
      m.append(i)
    else:
      m.append(' ')
  s2 = ''.join(m)
  s2 = re.sub(" +", " ", s2)
  return s2

def find_salary(text):
  if False:
    return (-1, -1)
  m = []
  for i in clear(text).split(' '):
    if i != '':
      if 400000 > int(i) > 10000 and (int(i) % 10) == 0:
        m.append(int(i))
  if m == []:
    return (-1, -1)
  if min(m) != max(m):
    return (min(m), max(m))
  return (min(m), -1)

In [ ]:
test['salary_from'] = 0 #поиск зарплат в описании
for i in range(len(test)):
  a = find_salary(test['description'].iloc[i])
  test.loc[i, 'salary_from'] = a[0]
  test.loc[i, 'salary_to'] = a[1]
test

,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,is_male,driving_license,region_id,company_id,city_id,target_prof,salary_from,salary_to,region_id_i,sal_class_tp
0,286961,водитель курьер доставка еда,3,<strong>Приглашаем на работу ВОДИТЕЛЕЙ-КУРЬЕРО...,0,18.0,50.0,False,True,-100.0,NaN,['B'],209,1172596.0,57,водитель,-1,-1,5.0,5.0
1,423834,упаковщик печение (вахта),7,"Прямой работодатель, с сезонным расширением пр...",0,NaN,NaN,False,False,NaN,NaN,[],3,1879927.0,1,упаковщик,-1,-1,6.0,4.0
2,336464,менеджер туризм,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,4,NaN,NaN,False,False,1.0,NaN,[],1278,2724192.0,57,менеджер,-1,-1,5.0,3.0
3,287634,повар сушистый,1,Компания PIZZASUSHIWOK приглашает на работу по...,0,18.0,50.0,False,True,-100.0,NaN,[],3,1172596.0,1,повар,-1,-1,6.0,3.0
4,174883,уборщик/уборщица азс 145,3,<br /> <br />Требуется уборщик/ца на АЗС 145.<...,0,NaN,NaN,False,True,-1.0,NaN,[],1278,1952643.0,1,уборщик,-1,-1,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14069,290923,маркировщик/ца детский товар,7,<strong>Работа вахтой на производственно-склад...,0,NaN,NaN,False,False,NaN,NaN,[],3,1709808.0,1,маркировщик,-1,-1,6.0,3.0
14070,175080,упаковщик чайный фабрика (вахта),7,На чайную фабрику в Москве требуются сотрудник...,0,NaN,NaN,False,False,NaN,NaN,[],3,1724853.0,1,упаковщик,-1,-1,6.0,4.0
14071,451820,главный бухгалтер тэо,6,Стабильно работающая более 10 лет компания ище...,4,NaN,NaN,False,True,3.0,NaN,[],4,2961361.0,2,бухгалтер,-1,-1,5.0,3.0
14072,257668,помощник руководитель,1,<br />\nТребования: <br>В связи с формирование...,0,NaN,NaN,False,False,NaN,NaN,[],3,NaN,1,ассистент,-1,-1,6.0,3.0


In [ ]:
columns = ['sal_class_tp','region_id_i', 'offer_experience_year_count', 'offer_education_id', 'operating_schedule_id', 'salary_from', 'salary_to'] #признаки, которые будут использоваться для предсказания зарплат

In [ ]:
#датасет для предсказания зарплат
new_train = train[columns]
new_train

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,3.0,5.0,1.0,0,1,34000,41000.0
1,4.0,6.0,NaN,0,7,66000,NaN
2,3.0,6.0,NaN,0,1,40000,40000.0
3,4.0,5.0,-100.0,0,7,35000,40000.0
4,2.0,3.0,2.0,4,1,30000,70000.0
...,...,...,...,...,...,...,...
56292,3.0,5.0,2.0,0,1,80000,NaN
56293,3.0,10.0,3.0,2,7,0,73000.0
56294,4.0,5.0,NaN,0,1,-1,-1.0
56295,4.0,3.0,2.0,4,1,-1,-1.0


In [ ]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56297 entries, 0 to 56296
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sal_class_tp                 56297 non-null  float64
 1   region_id_i                  56297 non-null  float64
 2   offer_experience_year_count  38622 non-null  float64
 3   offer_education_id           56297 non-null  int64  
 4   operating_schedule_id        56297 non-null  int64  
 5   salary_from                  56297 non-null  int64  
 6   salary_to                    34829 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 3.0 MB


In [ ]:
new_train

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,3.0,5.0,1.0,0,1,34000,41000.0
1,4.0,6.0,NaN,0,7,66000,NaN
2,3.0,6.0,NaN,0,1,40000,40000.0
3,4.0,5.0,-100.0,0,7,35000,40000.0
4,2.0,3.0,2.0,4,1,30000,70000.0
...,...,...,...,...,...,...,...
56292,3.0,5.0,2.0,0,1,80000,NaN
56293,3.0,10.0,3.0,2,7,0,73000.0
56294,4.0,5.0,NaN,0,1,-1,-1.0
56295,4.0,3.0,2.0,4,1,-1,-1.0


In [ ]:
new_train = new_train[new_train['region_id_i'].notna()]
new_train = new_train[new_train['salary_to'].notna()]
new_train = new_train[new_train['salary_from'].notna()]
new_train = new_train[new_train['offer_experience_year_count'].notna()]

In [ ]:
new_train

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,3.0,5.0,1.0,0,1,34000,41000.0
3,4.0,5.0,-100.0,0,7,35000,40000.0
4,2.0,3.0,2.0,4,1,30000,70000.0
9,5.0,5.0,-100.0,0,2,0,85000.0
12,3.0,4.0,-100.0,0,1,15280,18280.0
...,...,...,...,...,...,...,...
56284,2.0,5.0,-100.0,1,3,32800,36037.0
56287,3.0,6.0,-100.0,0,2,32000,64000.0
56291,2.0,6.0,0.0,0,3,37800,60000.0
56293,3.0,10.0,3.0,2,7,0,73000.0


In [ ]:
new_train = new_train.drop(new_train[new_train.salary_from < 1].index)
new_train = new_train.drop(new_train[new_train.salary_to < 1].index)
new_train = new_train.drop(new_train[new_train.region_id_i < 0].index)
new_train = new_train.drop(new_train[new_train.offer_experience_year_count < 0].index)

In [ ]:
new_train #обработанный датасет

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,3.0,5.0,1.0,0,1,34000,41000.0
4,2.0,3.0,2.0,4,1,30000,70000.0
14,5.0,6.0,1.0,0,2,80000,160000.0
17,4.0,5.0,1.0,0,7,40000,46000.0
33,2.0,4.0,1.0,0,3,30000,120000.0
...,...,...,...,...,...,...,...
56214,4.0,5.0,3.0,2,1,70000,80000.0
56224,4.0,4.0,1.0,4,1,45000,50000.0
56241,7.0,12.0,3.0,0,7,120000,140000.0
56276,2.0,6.0,1.0,0,1,45000,75000.0


In [ ]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(new_train[['sal_class_tp','region_id_i', 'offer_experience_year_count', 'offer_education_id', 'operating_schedule_id']], new_train['salary_to'], train_size=0.90) #разбиваем на тренировочную и тестовую выборку
X_train2, X_val2, y_train2, y_val2 = train_test_split(new_train[['sal_class_tp','region_id_i', 'offer_experience_year_count', 'offer_education_id', 'operating_schedule_id']], new_train['salary_from'], train_size=0.90)

In [ ]:
X_train1

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id
36996,4.0,6.0,3.0,4,1
31639,4.0,6.0,1.0,2,3
38462,2.0,4.0,1.0,0,1
9074,2.0,6.0,0.0,0,5
41713,4.0,5.0,3.0,0,1
...,...,...,...,...,...
16598,6.0,6.0,5.0,0,1
30193,4.0,6.0,1.0,0,1
29314,3.0,5.0,0.0,0,3
10171,3.0,4.0,5.0,0,3


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model_to = CatBoostRegressor() #обучаем модели

model_to.fit(X_train1, y_train1)
model_from = CatBoostRegressor()

model_from.fit(X_train2, y_train2)

Learning rate set to 0.056383
0:	learn: 43664.7244743	total: 47.7ms	remaining: 47.6s
1:	learn: 43218.7246137	total: 49.3ms	remaining: 24.6s
2:	learn: 42785.3033655	total: 50.7ms	remaining: 16.8s
3:	learn: 42414.1011662	total: 51.9ms	remaining: 12.9s
4:	learn: 42097.5254551	total: 53.2ms	remaining: 10.6s
5:	learn: 41750.6171059	total: 54.4ms	remaining: 9.02s
6:	learn: 41445.1254784	total: 55.8ms	remaining: 7.92s
7:	learn: 41180.8126374	total: 57.1ms	remaining: 7.08s
8:	learn: 40944.4855480	total: 58.3ms	remaining: 6.42s
9:	learn: 40724.9048034	total: 59.6ms	remaining: 5.9s
10:	learn: 40555.0242097	total: 60.7ms	remaining: 5.46s
11:	learn: 40357.8320820	total: 61.9ms	remaining: 5.1s
12:	learn: 40182.1893835	total: 63.2ms	remaining: 4.79s
13:	learn: 40028.9847351	total: 64.5ms	remaining: 4.54s
14:	learn: 39880.9390372	total: 65.7ms	remaining: 4.31s
15:	learn: 39759.9892933	total: 66.9ms	remaining: 4.12s
16:	learn: 39628.5829096	total: 68.3ms	remaining: 3.95s
17:	learn: 39534.8613023	total

In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
MAPE(y_val1, model_to.predict(X_val1)) #оцениваем точность модели


43.79849351240408

In [ ]:
MAPE(y_val2, model_from.predict(X_val2)) #оцениваем точность модели

31.83248934099021

In [ ]:
to_pred = test[columns]
to_pred

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,5.0,5.0,-100.0,0,3,-1,-1
1,4.0,6.0,NaN,0,7,-1,-1
2,3.0,5.0,1.0,4,3,-1,-1
3,3.0,6.0,-100.0,0,1,-1,-1
4,2.0,5.0,-1.0,0,3,-1,-1
...,...,...,...,...,...,...,...
14069,3.0,6.0,NaN,0,7,-1,-1
14070,4.0,6.0,NaN,0,7,-1,-1
14071,3.0,5.0,3.0,4,6,-1,-1
14072,3.0,6.0,NaN,0,1,-1,-1


In [ ]:
to_pred = to_pred.fillna(0)
to_pred[to_pred['offer_experience_year_count'] < 0] = 0

In [ ]:
to_pred

,sal_class_tp,region_id_i,offer_experience_year_count,offer_education_id,operating_schedule_id,salary_from,salary_to
0,0.0,0.0,0.0,0,0,0,0
1,4.0,6.0,0.0,0,7,-1,-1
2,3.0,5.0,1.0,4,3,-1,-1
3,0.0,0.0,0.0,0,0,0,0
4,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...
14069,3.0,6.0,0.0,0,7,-1,-1
14070,4.0,6.0,0.0,0,7,-1,-1
14071,3.0,5.0,3.0,4,6,-1,-1
14072,3.0,6.0,0.0,0,1,-1,-1


In [ ]:
test['salary_form_cat'] = model_from.predict(to_pred.drop(['salary_from',	'salary_to'], axis = 1))
test['salary_to_cat'] = model_to.predict(to_pred.drop(['salary_from',	'salary_to'], axis = 1))

In [ ]:
import pickle
with open('cat_to.pickle', 'wb') as f:
  pickle.dump(model_to, f)
with open('cat_from.pickle', 'wb') as f:
  pickle.dump(model_to, f)

In [ ]:
for i in range(len(test)):
  if not test['salary_from'].iloc[i] > 0:
    test.loc[i, 'salary_from'] = test['salary_form_cat'].iloc[i]
  if not test['salary_to'].iloc[i] > 0:
    test.loc[i, 'salary_to'] = test['salary_to_cat'].iloc[i]

In [ ]:
test[test['salary_from'] != test['salary_form_cat']]

,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,...,region_id,company_id,city_id,target_prof,salary_from,salary_to,region_id_i,sal_class_tp,salary_form_cat,salary_to_cat
5,276796,почтальон,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,1,18.0,NaN,False,False,-100.0,...,60,1268027.0,38,почтальон,13000.0,38753.718568,4.0,1.0,10282.202679,38753.718568
12,287682,охранник (вахта),7,<strong>Вакансия компании ООО ЧОП Щит</strong>...,0,NaN,NaN,False,True,-1.0,...,3,1636491.0,259,охранник,24000.0,51000.000000,6.0,2.0,10282.202679,38753.718568
18,56390,уборщик (клинер),2,<strong>Вакансия компании Клининг Счастья ( ИП...,0,NaN,NaN,False,True,-1.0,...,299,1636491.0,8,уборщик,25000.0,45000.000000,4.0,2.0,10282.202679,38753.718568
26,151566,сварщик полуавтомат,7,&nbsp;<br />\r\n\r\n\r\n<strong>Обязанности:</...,0,18.0,60.0,True,True,1.0,...,621,2162445.0,49,сварщик,75000.0,103632.368214,10.0,7.0,87857.934907,103632.368214
41,346294,продавец консультант магазин куулклевер. мясно...,3,"<strong>КуулКлевер &ndash; это место, где вам ...",0,NaN,NaN,False,False,NaN,...,3,266870.0,1,консультант,45000.0,110000.000000,6.0,2.0,33008.149617,45579.251232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14053,439484,мерчендайзер (м. тёплый стан),5,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,0,NaN,NaN,False,False,-100.0,...,3,344421.0,1,мерчендайзер,15900.0,38753.718568,6.0,2.0,10282.202679,38753.718568
14056,105962,помощник повар ( сотрудник раздача столовой),3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,0,NaN,NaN,False,True,1.0,...,4548,1882525.0,35,сотрудник,25000.0,63602.175020,18.0,3.0,37933.379465,63602.175020
14057,108793,охранник,7,<strong>Охранник на парковку в Москве</strong>...,0,18.0,55.0,False,True,-100.0,...,3,253590.0,174,охранник,57000.0,38753.718568,6.0,2.0,10282.202679,38753.718568
14065,56768,водитель категория b,2,Сервис такси приглашает водителей на работу. С...,0,21.0,NaN,True,True,-100.0,...,104,2103886.0,7,водитель,80000.0,38753.718568,4.0,5.0,10282.202679,38753.718568


In [ ]:
test

,id,custom_position,operating_schedule_id,description,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,...,region_id,company_id,city_id,target_prof,salary_from,salary_to,region_id_i,sal_class_tp,salary_form_cat,salary_to_cat
0,286961,водитель курьер доставка еда,3,<strong>Приглашаем на работу ВОДИТЕЛЕЙ-КУРЬЕРО...,0,18.0,50.0,False,True,-100.0,...,209,1172596.0,57,водитель,10282.202679,38753.718568,5.0,5.0,10282.202679,38753.718568
1,423834,упаковщик печение (вахта),7,"Прямой работодатель, с сезонным расширением пр...",0,NaN,NaN,False,False,NaN,...,3,1879927.0,1,упаковщик,56994.146743,78071.977113,6.0,4.0,56994.146743,78071.977113
2,336464,менеджер туризм,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,4,NaN,NaN,False,False,1.0,...,1278,2724192.0,57,менеджер,41215.190199,68122.573329,5.0,3.0,41215.190199,68122.573329
3,287634,повар сушистый,1,Компания PIZZASUSHIWOK приглашает на работу по...,0,18.0,50.0,False,True,-100.0,...,3,1172596.0,1,повар,10282.202679,38753.718568,6.0,3.0,10282.202679,38753.718568
4,174883,уборщик/уборщица азс 145,3,<br /> <br />Требуется уборщик/ца на АЗС 145.<...,0,NaN,NaN,False,True,-1.0,...,1278,1952643.0,1,уборщик,10282.202679,38753.718568,5.0,2.0,10282.202679,38753.718568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14069,290923,маркировщик/ца детский товар,7,<strong>Работа вахтой на производственно-склад...,0,NaN,NaN,False,False,NaN,...,3,1709808.0,1,маркировщик,47125.884190,71046.384348,6.0,3.0,47125.884190,71046.384348
14070,175080,упаковщик чайный фабрика (вахта),7,На чайную фабрику в Москве требуются сотрудник...,0,NaN,NaN,False,False,NaN,...,3,1724853.0,1,упаковщик,56994.146743,78071.977113,6.0,4.0,56994.146743,78071.977113
14071,451820,главный бухгалтер тэо,6,Стабильно работающая более 10 лет компания ище...,4,NaN,NaN,False,True,3.0,...,4,2961361.0,2,бухгалтер,38858.630220,72870.840954,5.0,3.0,38858.630220,72870.840954
14072,257668,помощник руководитель,1,<br />\nТребования: <br>В связи с формирование...,0,NaN,NaN,False,False,NaN,...,3,NaN,1,ассистент,40666.058869,55268.768601,6.0,3.0,40666.058869,55268.768601


In [ ]:
result = pd.DataFrame()
result['id'] = test['id']
result['salary_from'] = test['salary_from']
result['salary_to'] = test['salary_to']
result['target_prof'] = test['target_prof']
result

,id,salary_from,salary_to,target_prof
0,286961,10282.202679,38753.718568,водитель
1,423834,56994.146743,78071.977113,упаковщик
2,336464,41215.190199,68122.573329,менеджер
3,287634,10282.202679,38753.718568,повар
4,174883,10282.202679,38753.718568,уборщик
...,...,...,...,...
14069,290923,47125.884190,71046.384348,маркировщик
14070,175080,56994.146743,78071.977113,упаковщик
14071,451820,38858.630220,72870.840954,бухгалтер
14072,257668,40666.058869,55268.768601,ассистент


In [ ]:
result.to_csv('result12.csv')

In [ ]:
def pred(s):
  s = np.array(vectorizer.transform([s]).todense(), dtype=np.float16)
  return modelq.predict(s)[0]



In [ ]:
pred('водитель2')

'витражист'